In [197]:

# import required module
import os
import pandas as pd
import numpy as np

import country_converter as coco
cc = coco.CountryConverter()

In [198]:
##Add GDP per capita before removing years
GDPpc_Data = (pd.read_excel("../mpd2020.xlsx", sheet_name = "Full data")
                .drop(columns = ["pop", "countrycode"])
                .rename(columns = {"gdppc" : "GDPPC", "country" : "Country", "year" : "Year"})
                .replace({'United States' : 'USA'}))
#GDPpc_Data.head()
GDPpc_Data["Country"] = cc.convert(names = GDPpc_Data['Country'], to = "name_short")
GDPpc_Data.GDPPC =  pd.to_numeric(GDPpc_Data.GDPPC)  #Convert column to numeric for calculations

D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D.R. of the Congo not found in regex
D

In [199]:
GDPpc_Data

,Country,Year,GDPPC
0,Afghanistan,1820,NaN
1,Afghanistan,1870,NaN
2,Afghanistan,1913,NaN
3,Afghanistan,1950,1156.0000
4,Afghanistan,1951,1170.0000
...,...,...,...
21677,Zimbabwe,2014,1594.0000
21678,Zimbabwe,2015,1560.0000
21679,Zimbabwe,2016,1534.0000
21680,Zimbabwe,2017,1582.3662


In [200]:
TWWI = pd.DataFrame(columns=['Country', 'Year', 'TWWI'])

# assign directory
directory = 'DOTdata'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
#for i in range(1,187):
    #f = f"DOTdata\Exports_to_Count ({i}).xlsx"
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        
        #---------------------------------------------------------------------------------------------
        Coun = ( pd.read_excel(f, skiprows = 3, nrows = 0, usecols = 'B')
                .columns[0] ) #gets country name only
        Coun = cc.convert(names = Coun, to = "name_short")

        temp = ( pd.read_excel(f, skiprows = 6)
                .drop(columns = ["Unnamed: 0"])
                .rename(columns= {'Unnamed: 1': 'Country'})
                .fillna(0) ) #gets data

        #Coun
        temp = temp[temp['Country']!= "Aruba, Kingdom of the Netherlands"]
        temp = temp[temp['Country']!= "Curaçao, Kingdom of the Netherlands"] #countries that don't matter
        temp = temp[temp['Country']!= "Sint Maarten, Kingdom of the Netherlands"]

        temp.Country = cc.convert(names = temp['Country'], to = "name_short")
        temp = temp[temp["Country"] != "not found"]
        temp = temp.melt(id_vars='Country', var_name="Year", value_name="ExpRec")
        temp.Year = temp.Year.astype(int)
        temp.ExpRec = temp.apply(lambda x: float(x['ExpRec'].split(" ")[0].replace(',','')) if type(x['ExpRec']) == str else x['ExpRec'], axis=1) ## 1.2 e issue solving

        CounGDP = GDPpc_Data[GDPpc_Data.Country == Coun]
        temp2 = temp.merge(CounGDP[['Year', 'GDPPC']], on ='Year', how = 'left').rename(columns = {'GDPPC': 'DomGDPPC'})
        temp2['w'] =  temp2.apply(lambda x: np.nan if ( np.isnan(x['DomGDPPC']) or x['DomGDPPC'] == 0 ) else x['ExpRec'] / x['DomGDPPC'] * 1e6 , axis=1)  ##calculate w per year
        temp2 = temp2.sort_values(by = ["Country", "Year"])

        TW = ( temp2.groupby('Country')[["Year","w"]]
            .rolling(5, min_periods=5, on='Year', closed = "left")
            .mean().reset_index().drop(columns = "level_1")
            .rename(columns = {'w':'TW'}).fillna(0) ) ##calculating TW

        temp3 = temp2.merge(TW, on= ['Year','Country'], how = "outer").merge(GDPpc_Data, on = ["Country", "Year"], how = "left")
        temp3['TWi'] = temp3.apply(lambda x: x['TW'] * x['GDPPC'], axis = 1)
        #temp3.tail(50)

        temp4 = temp3.groupby('Year')['TWi'].sum().reset_index().rename(columns = {"TWi": 'TWWI'})
        temp4['Country'] = Coun
        TWWI = pd.concat([TWWI, temp4])
#==============================================================================

Belgium-Luxembourg not found in regex
Netherlands Antilles not found in regex
Serbia and Montenegro not found in regex
Yugoslavia not found in regex
Netherlands Antilles not found in regex
Serbia and Montenegro not found in regex
Belgium-Luxembourg not found in regex
Eastern Germany not found in regex
Netherlands Antilles not found in regex
Serbia and Montenegro not found in regex
Yugoslavia not found in regex
Netherlands Antilles not found in regex
Netherlands Antilles not found in regex
Serbia and Montenegro not found in regex
Belgium-Luxembourg not found in regex
Eastern Germany not found in regex
Netherlands Antilles not found in regex
Serbia and Montenegro not found in regex
Yugoslavia not found in regex
Belgium-Luxembourg not found in regex
Serbia and Montenegro not found in regex
Yugoslavia not found in regex
Belgium-Luxembourg not found in regex
Eastern Germany not found in regex
Netherlands Antilles not found in regex
Serbia and Montenegro not found in regex
Yugoslavia not fou

In [201]:
TWWI.to_csv("TWWI.csv", index = False)

In [202]:
f = "DOTdata\Exports_to_Count (99).xlsx"

Coun = ( pd.read_excel(f, skiprows = 3, nrows = 0, usecols = 'B')
        .columns[0] ) #gets country name only
Coun = cc.convert(names = Coun, to = "name_short")

temp = ( pd.read_excel(f, skiprows = 6)
        .drop(columns = ["Unnamed: 0"])
        .rename(columns= {'Unnamed: 1': 'Country'})
        .fillna(0) ) #gets data

#Coun
temp = temp[temp['Country']!= "Aruba, Kingdom of the Netherlands"]
temp = temp[temp['Country']!= "Curaçao, Kingdom of the Netherlands"] #countries that don't matter
temp = temp[temp['Country']!= "Sint Maarten, Kingdom of the Netherlands"]

temp.Country = cc.convert(names = temp['Country'], to = "name_short")
temp = temp[temp["Country"] != "not found"]
temp = temp.melt(id_vars='Country', var_name="Year", value_name="ExpRec")
temp.Year = temp.Year.astype(int)
temp.ExpRec = temp.apply(lambda x: float(x['ExpRec'].split(" ")[0].replace(',','')) if type(x['ExpRec']) == str else x['ExpRec'], axis=1) ## 1.2 e issue solving

CounGDP = GDPpc_Data[GDPpc_Data.Country == Coun]
temp2 = temp.merge(CounGDP[['Year', 'GDPPC']], on ='Year', how = 'left').rename(columns = {'GDPPC': 'DomGDPPC'})
temp2['w'] =  temp2.apply(lambda x: np.nan if ( np.isnan(x['DomGDPPC']) or x['DomGDPPC'] == 0 ) else x['ExpRec'] / x['DomGDPPC'] * 1e6 , axis=1)  ##calculate w per year
temp2 = temp2.sort_values(by = ["Country", "Year"])

TW = ( temp2.groupby('Country')[["Year","w"]]
      .rolling(5, min_periods=5, on='Year', closed = "left")
      .mean().reset_index().drop(columns = "level_1")
      .rename(columns = {'w':'TW'}).fillna(0) ) ##calculating TW

temp3 = temp2.merge(TW, on= ['Year','Country'], how = "outer").merge(GDPpc_Data, on = ["Country", "Year"], how = "left")
temp3['TWi'] = temp3.apply(lambda x: np.nan if ( np.isnan(x['TW']) or np.isnan(x['GDPPC']) ) else x['TW'] * x['GDPPC'], axis = 1)
#temp3.tail(50)

temp4 = temp3.groupby('Year')['TWi'].sum().reset_index().rename(columns = {"TWi": 'TWWI'})
temp4['Country'] = Coun
temp3

Belgium-Luxembourg not found in regex
Eastern Germany not found in regex
Netherlands Antilles not found in regex
Serbia and Montenegro not found in regex
Yugoslavia not found in regex


,Country,Year,ExpRec,DomGDPPC,w,TW,GDPPC,TWi
0,Afghanistan,1961,0.000000,2868.0000,0.000000,0.000000,1309.0000,0.000000
1,Afghanistan,1966,0.000000,2750.0000,0.000000,0.000000,1272.0000,0.000000
2,Afghanistan,1967,0.000000,2907.0000,0.000000,0.000000,1277.0000,0.000000
3,Afghanistan,1968,0.000000,3151.0000,0.000000,0.000000,1290.0000,0.000000
4,Afghanistan,1969,0.000000,3355.0000,0.000000,0.000000,1278.0000,0.000000
...,...,...,...,...,...,...,...,...
10907,Zimbabwe,2016,0.000000,14331.0000,0.000000,0.020987,1534.0000,32.193424
10908,Zimbabwe,2017,0.000074,14267.2306,0.005187,0.020987,1582.3662,33.208466
10909,Zimbabwe,2018,0.000000,14228.0250,0.000000,0.001215,1611.4052,1.958178
10910,Zimbabwe,2019,0.148030,NaN,NaN,0.001037,NaN,NaN
